In [1]:
import os
import torch
import numpy as np
import pandas as pd
import scanpy as sc
from scipy import sparse
import tqdm
from torch.utils.data import Dataset
import pickle

In [3]:
#data_dir = "../data/rh2_cf5_tcell_type16_veh"
data_dir = "../data/rh2_cf5_tcell_dim22_allcond_imp_v_noimp"
load_ct=True
keep_sparse=False

In [4]:
genes = open(os.path.join(data_dir, "genes.txt")).read().strip().split("\n") #done
barcodes = open(os.path.join(data_dir, "barcodes.txt")).read().strip().split("\n") #done
meta = pd.read_csv(os.path.join(data_dir, "meta.csv"), sep=",") #done
cell_types = pd.read_csv(os.path.join(data_dir, "cell_types.txt"), 
                         sep="\t").reset_index(drop=True)["Category"]  #done

ct_id = sorted(set(cell_types))
mapping_ct = {c:idx for idx, c in enumerate(ct_id)}

X = []
y = []
ct = []

adata_np_array = np.genfromtxt(os.path.join(data_dir, "RawCounts.csv"), delimiter=',',dtype="float32") #done
adata = sc.AnnData(adata_np_array, obs=barcodes ,var=genes) #done

barcodes = adata.obs[0].tolist()  #good
genes = adata.var[0].tolist()  #good

#adata = sc.AnnData(dat.astype(np.float32), obs=barcodes, var=genes)
# adata = sc.AnnData(dat.astype(np.float32))
# before: (32588, 32871) | after: (32588, 29696)
#sc.pp.filter_genes(adata, min_cells=5)
#sc.pp.normalize_total(adata, target_sum=1e4)
# sc.pp.scale(adata, max_value=10, zero_center=True)

# if keep_sparse is False:
#     adata.X = adata.X.toarray()

for ind in tqdm.tqdm(sorted(set(meta.donor_id))):
    disease = list(set(meta.disease__ontology_label[meta.donor_id == ind]))
    assert len(disease) == 1
    if disease[0] == "long COVID-19" or disease[0] == "respiratory failure":
        continue
    x = adata.X[meta.donor_id == ind]
    X.append(x)
    y.append(disease[0])
    ct.append([mapping_ct[c] for c in cell_types[meta.donor_id == ind]])

class_id = sorted(set(y))
mapping = {c:idx for idx, c in enumerate(class_id)}
y = [mapping[c] for c in y]
 
# [Size of dataset] COVID-19: 35 | long COVID-19: 2 | normal: 15 | respiratory failure: 6
print(("[Size of dataset] "+" | ".join(["{:s}: {:d}"] * len(class_id))).format(*[item for i in range(len(class_id)) for item in [class_id[i], y.count(i)]]))



100%|██████████| 16/16 [00:05<00:00,  2.82it/s]

[Size of dataset] IMPRV: 8 | NoIMP: 8


In [5]:
from load_data_cytof_ver013 import OurDataset
data_for_outside = OurDataset(X=X, y=y, cell_id=barcodes, gene_id=genes, class_id=class_id, ct=ct, ct_id=ct_id)

In [6]:
type(data_for_outside)

load_data_cytof_ver013.OurDataset

In [7]:
import pickle
with open('alldata_from_load_data.pkl', 'wb') as file: 
    pickle.dump(data_for_outside, file)

In [8]:
import pickle
with open('alldata_from_load_data.pkl', 'rb') as file: 
    alldata_from_load_data = pickle.load(file)
    
type(alldata_from_load_data)

load_data_cytof_ver013.OurDataset

In [9]:
type(data_for_outside[0]), len(data_for_outside[0])

(tuple, 3)

In [ ]:
type(data_for_outside.X)

In [ ]:
type(data_for_outside.X),len(data_for_outside.X), type(data_for_outside.y),len(data_for_outside.y) 

In [ ]:
type(data_for_outside.ct),len(data_for_outside.ct) 

In [ ]:
data_for_outside.X[1].shape

In [10]:
from sklearn.model_selection import train_test_split

In [11]:
# self.train_set, self.test_set
seed_numb = 123
torch.manual_seed(seed_numb)
torch.cuda.manual_seed(seed_numb)
split_ratio = [0.5, 0.25, 0.25]
#all_data, test_data = train_test_split(data_for_outside, test_size=0.25, shuffle=False, random_state=seed_numb)
train_set, test_set = train_test_split(data_for_outside, test_size=split_ratio[2], random_state=seed_numb)
train_set, val_set = train_test_split(train_set, test_size=split_ratio[1] / (1-split_ratio[2]), random_state=seed_numb)

In [12]:
len(train_set)

8

In [13]:
for ii in range(len(train_set)):
    print(train_set[ii][0].shape[0])
    # total_sum = total_sum + all_data[ii][0].shape[0]

# print(total_sum)

78892
76057
61938
124755
223449
223162
25799
102217


In [14]:
for ii in range(len(val_set)):
    print(val_set[ii][0].shape[0])
    # total_sum = total_sum + all_data[ii][0].shape[0]

# print(total_sum)

99436
93515
111811
54582


In [15]:

for ii in range(len(test_set)):
    print(test_set[ii][0].shape[0])
    # total_sum = total_sum + all_data[ii][0].shape[0]

# print(total_sum)

17351
156563
153867
118884


In [16]:
# seed_numb = 123
# split_ratio = [0.55, 0.225, 0.225]

trainset_fn = 'train_set_'+str(seed_numb)+'_'+str(split_ratio[0])+'.pkl'
valset_fn = 'val_set_'+str(seed_numb)+'_'+str(split_ratio[0])+'.pkl'
testset_fn = 'test_set_'+str(seed_numb)+'_'+str(split_ratio[0])+'.pkl'

print(trainset_fn, valset_fn, testset_fn)


train_set_123_0.5.pkl val_set_123_0.5.pkl test_set_123_0.5.pkl


In [17]:
import pickle

with open(trainset_fn, 'wb') as file: 
    pickle.dump(train_set, file)

with open(valset_fn, 'wb') as file: 
    pickle.dump(val_set, file)

with open(testset_fn, 'wb') as file: 
    pickle.dump(test_set, file)

In [ ]:
import pickle
with open(trainset_fn, 'rb') as file:   #Myles added
    train_set = pickle.load(file)    
    
with open(valset_fn, 'rb') as file:   #Myles added
    val_set = pickle.load(file)    
        
with open(testset_fn, 'rb') as file:   #Myles added
    test_set = pickle.load(file)    

In [18]:
len(train_set),len(val_set), len(test_set)

(8, 4, 4)

In [19]:
all_data = train_set

In [20]:
for ii in range(len(val_set)):
  all_data.append(val_set[ii])  
# all_data.append(val_set[0])
# all_data.append(val_set[1])
# all_data.append(val_set[2])
# all_data.append(val_set[3])
# all_data.append(val_set[4])

len(all_data)

12

In [21]:
for ii in range(len(test_set)):
      all_data.append(test_set[ii])  
# all_data.append(val_set[0])
# all_data.append(val_set[1])
# all_data.append(val_set[2])
# all_data.append(val_set[3])
# all_data.append(val_set[4])

len(all_data)

16

In [22]:
with open('all_data_123_0.5.pkl', 'wb') as file: 
    pickle.dump(all_data, file)

In [3]:
for ii in range(16):
    print(all_data[ii][0].shape[0])
    # total_sum = total_sum + all_data[ii][0].shape[0]

# print(total_sum)

78892
76057
61938
124755
223449
223162
25799
102217
99436
93515
111811
54582
17351
156563
153867
118884


In [2]:
with open('all_data_123_0.5.pkl', 'rb') as file: 
    all_data = pickle.load(file)

In [3]:
selected_data = []
selected_data.append(all_data[5])
selected_data.append(all_data[8])
selected_data.append(all_data[13])

In [4]:
selected_data[0][0]. shape,selected_data[1][0].shape, selected_data[2][0].shape

((223162, 22), (99436, 22), (156563, 22))

In [10]:
with open('RH2_selected_1034_1007_1072.pkl', 'wb') as file: 
    pickle.dump(selected_data, file)

In [4]:
type(all_data[15])

tuple

In [5]:
type(all_data)

list

In [ ]:
len(all_data)

In [ ]:
for ii in range(16):
    print(all_data[ii][1])

In [ ]:
del all_data

In [ ]:
import pickle
with open('all_data_hf_il1b_week0.pkl', 'rb') as file: 
    all_data = pickle.load(file)

In [ ]:
len(test_data[0])

In [ ]:
test_data[0][1]

In [ ]:
test_data[0]

In [ ]:
all_data[15]

In [ ]:
len(all_data),len(test_data)

In [ ]:
alll_data = [all_data, test_data]

In [ ]:
len(alll_data)

In [ ]:
len(data_for_outside.X)
data_for_outside.X[0].shape, type(data_for_outside.X[0])

In [ ]:
# device = "cuda:0"
device = "cpu"
stacked_X = torch.cat([torch.tensor(data_for_outside.X[i]) for i in range(24)]).to(device)

In [ ]:
type(stacked_X), stacked_X.shape

In [ ]:
%store -r model_copy
from torchinfo import summary
summary(model_copy)

In [ ]:
# Loading best or last model
model_copy.load_state_dict(torch.load("../checkpoint/rh2_cf5_tcell_type16_veh/protocell_rh2_cf5_tcell_type16_veh_16_8_testing_w_cpu/last_model.pt"))
model_copy.eval()
#config.model.load_state_dict(torch.load("../checkpoint/rh2_1mil_16d_veh/protocell_rh2_dim16_veh_wo_pretrain_123_latent_reconstruct_16_8/best_model.pt"))

In [ ]:
# for p in model_copy.parameters():
#     if p.requires_grad:
#          print(p.name, p.data)

In [ ]:
stacked_X_tensor_latent_space = model_copy.encode(stacked_X)

In [ ]:
type(stacked_X_tensor_latent_space), stacked_X_tensor_latent_space.shape

In [ ]:
array_latent_space = stacked_X_tensor_latent_space.cpu().detach().numpy()

In [ ]:
array_latent_space.shape, type(array_latent_space)

In [ ]:
np.savetxt("array_latent_space.csv",array_latent_space, delimiter=",")import umap

In [ ]:
import umap

In [ ]:
embedding = umap.UMAP(n_neighbors=15, min_dist=0.01, metric='euclidean').fit_transform(array_latent_space) #[0:10000,0:8]

In [ ]:
import pickle
with open('umap_full.pkl', 'wb') as file: 
    # A new file will be created 
    pickle.dump(embedding, file)

In [ ]:
import pickle
with open('umap_full.pkl', 'rb') as file: 
    umap_load = pickle.load(file)

In [ ]:
# embedding.shape
umap_load.shape

In [ ]:
embedding[0:100:10,0]

In [ ]:
from matplotlib.pyplot import subplots
ax = subplots(figsize=(4,4))[1]
ax.scatter(embedding[0:1142025:100,0] , embedding[0:1142025:100,1], s = 2, c = 'r')
# ax.set_xlabel('Orginal');
# ax.set_ylabel('Reconstr');
# ax.set_title('Component '+str(component));

In [ ]:
import seaborn as sns
pal = sns.color_palette(['#e6194B', '#3cb44b', '#ffe119', '#4363d8', '#f58231', '#911eb4', '#42d4f4', 
                         '#f032e6', '#bfef45', '#469990', '#dcbeff', '#9A6324', '#800000', '#808000',
                         '#2f4f4f', '#a9a9a9', '#ffd8b1', '#000000', '#fffac8', '#aaffc3'])

In [ ]:
partition_table = cell_types_numbers.value_counts().sort_index().to_numpy()

partition = 0

index_start = 62903+25564 #np.sum(partition_table[0:(partition)])
index_end = 62903+25564+19745 #np.sum(partition_table[0:(partition+1)])
step = 5
p = sns.scatterplot(x=embedding[index_start:index_end:step,0], y=embedding[index_start:index_end:step,1], s=2, 
                    alpha=0.8, hue=cell_types_numbers_copy[index_start:index_end:step], palette = pal[0:15])#hue=label, s=s, palette=pal, 

In [ ]:
cell_types = pd.read_csv("../data/rh2_cf5_tcell_type16_veh/cell_types.txt", sep="\t").reset_index(drop=True)["Category"]  #done

In [ ]:
cell_types[0:10]

In [ ]:
type(cell_types)

In [ ]:
cell_types_numbers.query(Category=='cell_I')

In [ ]:
myseries[myseries == 7]

In [ ]:
cell_types_numbers[cell_types_numbers == 'cell_I']

In [ ]:
set(cell_types_numbers)

In [ ]:
cell_types_numbers

In [ ]:
cell_types_numbers.value_counts().sort()

In [ ]:
cell_types_numbers = cell_types
ii = 1
for cell_type_temp in sorted(set(cell_types)):
    cell_types_numbers[cell_types_numbers == cell_type_temp] = ii
    ii = ii + 1

In [ ]:
type(set(cell_types_numbers))

In [ ]:
sorted(set(cell_types_numbers))

In [ ]:
cell_types_numbers.value_counts()

In [ ]:
partition_table = cell_types_numbers.value_counts().sort_index().to_numpy()
partition_table

In [ ]:
type(partition_table)

In [ ]:
partition_table[0]

In [ ]:
np.sum(partition_table[0:(0)])

In [ ]:
meta = pd.read_csv("../data/rh2_cf5_tcell_type16_veh/meta.csv", sep=",") #done

In [ ]:
meta.shape

In [ ]:
meta.donor_id.value_counts().sort_index()

In [ ]:
#import umap
# import matplotlib.pyplot as plt
# import seaborn as sns

# from sklearn import datasets
# from sklearn.manifold import TSNE

# tsne = TSNE(n_components=2, random_state=42)
# tsne_coord = tsne.fit_transform(array_latent_space) 

In [ ]:
reconstructed_X = config.model.decode(config.model.encode(stacked_X))

In [ ]:
type(reconstructed_X), reconstructed_X.shape

In [ ]:
array_X = stacked_X.cpu().detach().numpy()
array_reconstructed_X = reconstructed_X.cpu().detach().numpy()

In [ ]:
array_X.shape, array_reconstructed_X.shape

In [ ]:
from matplotlib.pyplot import subplots

In [ ]:
end_idx = array_X.shape[0]

ax = subplots(figsize=(4,4))[1]
every = 500
component = 15
x_data = array_X[0:end_idx:every,component]
recon_x_data = array_reconstructed_X[0:end_idx:every,component]
ax.scatter(x_data , recon_x_data, s = 2, c = 'r')
ax.set_xlabel('Orginal');
ax.set_ylabel('Reconstr');
ax.set_title('Component '+str(component));

In [ ]:
temp_z1 = config.model.encode(torch.tensor(data_for_outside.X[1]))
temp_z1_nparray = temp_z1.detach().numpy()
df_temp_z1_nparray = pd.DataFrame(temp_z1_nparray)

In [ ]:
df_temp_z1_nparray.shape

In [ ]:
df = pd.concat([df_temp_z_nparray, df_temp_z1_nparray], ignore_index=True)

In [ ]:
df.shape

In [ ]:
type(temp_z)

In [ ]:
temp_x_hat = config.model.decode(temp_z)
temp_x_hat.shape

In [ ]:
temp_z_nparray.shape

In [ ]:
temp_z_nparray = temp_z_nparray[0:50,0:50]

In [ ]:
df_temp_z_nparray.shape

In [ ]:
df_temp_z_nparray

In [ ]:
import seaborn as sns

In [ ]:
column_headers = list(df_temp_z_nparray)
print("The Column Header :", column_headers)

In [ ]:
df_temp_z_nparray = df_temp_z_nparray.rename(columns={1: "one", 2: "two"})

In [ ]:
df_temp_z_nparray.head()

In [ ]:
import plotly.express as px

In [ ]:
fig = px.line(df_temp_z_nparray, x="one", y="two", title='Life expectancy in Canada')
fig.show()
# sns.lineplot(data=temp_z_nparray)

In [ ]:
from matplotlib.pyplot import subplots

In [ ]:
ax = subplots(figsize=(8,8))[1]
ax.scatter(df_temp_z_nparray.one, df_temp_z_nparray.two, s = 5, c = 'r')
ax.set_xlabel('Fitted value');
ax.set_ylabel('Residual');

In [ ]:
['x%d' % i for i in range(1, 6)]

In [ ]:
len(data_for_outside.ct[0])

In [ ]:
type(torch.tensor(data_for_outside.y[0]))

In [ ]:
len(data_for_outside.X[0])

In [ ]:
config.model(data_for_outside.X[0], torch.tensor(data_for_outside.y[0]),sparse=False)

In [ ]:
loss_test, logits_test, ct_logit_test = config.model(stacked_X, sparse=False) 

In [ ]:
from sklearn.model_selection import train_test_split
train_set_outside, test_set_outside = train_test_split(data_for_outside, test_size=1, random_state=123)

In [ ]:
len(train_set_outside)

In [ ]:
len(test_set_outside)

In [ ]:
test_set_outside